In [1]:
import os
import sys
!{sys.executable} -m pip install pydicom

import pydicom as dm
import shutil
import numpy as np
from xml.etree import ElementTree as ET
import matplotlib.pyplot as plt

!{sys.executable} -m pip install opencv-python

import cv2

!{sys.executable} -m pip install SimpleITK
import SimpleITK
from scipy import ndimage as ndi
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import disk, dilation, binary_erosion, binary_closing
from skimage.filters import roberts, sobel


NAMESPACE = "{http://www.nih.gov}"
OUTPUT_PATH = "./output/"
ROOT_PATH = '../../Downloads/LIDC-IDRI/'

# ROOT_PATH = './data/'
# OUTPUT_PATH = "./output2/"

patients= os.listdir(ROOT_PATH)
print(patients)

xml_lack_num = 0
ORIGIN_IMAGE_SIZE = 512

['LIDC-IDRI-0787', 'LIDC-IDRI-1003', 'LIDC-IDRI-0125', 'LIDC-IDRI-0919', 'LIDC-IDRI-0317', 'LIDC-IDRI-0773', 'LIDC-IDRI-0541', 'LIDC-IDRI-0328', 'LIDC-IDRI-0926', 'LIDC-IDRI-0546', 'LIDC-IDRI-0774', 'LIDC-IDRI-0310', 'LIDC-IDRI-0122', 'LIDC-IDRI-1004', 'LIDC-IDRI-0780', 'LIDC-IDRI-0921', 'LIDC-IDRI-0579', 'LIDC-IDRI-0326', 'LIDC-IDRI-0114', 'LIDC-IDRI-0928', 'LIDC-IDRI-0570', 'LIDC-IDRI-0742', 'LIDC-IDRI-0584', 'LIDC-IDRI-0917', 'LIDC-IDRI-0319', 'LIDC-IDRI-0789', 'LIDC-IDRI-0583', 'LIDC-IDRI-0745', 'LIDC-IDRI-0577', 'LIDC-IDRI-0113', 'LIDC-IDRI-0321', 'LIDC-IDRI-0910', 'LIDC-IDRI-0548', 'LIDC-IDRI-0944', 'LIDC-IDRI-0178', 'LIDC-IDRI-0381', 'LIDC-IDRI-0523', 'LIDC-IDRI-0711', 'LIDC-IDRI-0375', 'LIDC-IDRI-0147', 'LIDC-IDRI-0729', 'LIDC-IDRI-0943', 'LIDC-IDRI-0140', 'LIDC-IDRI-0372', 'LIDC-IDRI-0716', 'LIDC-IDRI-0524', 'LIDC-IDRI-0988', 'LIDC-IDRI-0386', 'LIDC-IDRI-0975', 'LIDC-IDRI-0149', 'LIDC-IDRI-0981', 'LIDC-IDRI-0720', 'LIDC-IDRI-0512', 'LIDC-IDRI-0176', 'LIDC-IDRI-0344', 'LIDC-IDR

In [2]:
def series_retrieval(path):
    xml_file = ""
    for i in os.listdir(path):
        if os.path.splitext(i)[-1] == ".xml":
            xml_file = os.path.join(path, i)
        if os.path.splitext(i)[-1] == ".dcm":
            i = os.path.join(path, i)
            ds = dm.read_file(i)
#             if ((ds.BodyPartExamined.upper() not in ('CHEST', 'LUNG')
#                  or ds.Modality.upper() != 'CT')):
            if ds.Modality.upper() != 'CT':
                return xml_file, False
            new_name = ds.SOPInstanceUID + '.dcm'
            os.rename(i, os.path.join(path, new_name))
    return xml_file, True

def baweraopen(image,size):
    '''
    @image:单通道二值图，数据类型uint8
    @size:欲去除区域大小(黑底上的白区域)
    '''
    output=image.copy()
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    for i in range(1,nlabels-1):
        regions_size=stats[i,4]
        if regions_size<size:
            x0=stats[i,0]
            y0=stats[i,1]
            x1=stats[i,0]+stats[i,2]
            y1=stats[i,1]+stats[i,3]
            for row in range(y0,y1):
                for col in range(x0,x1):
                    if labels[row,col]==i:
                        output[row,col]=0
    return output




In [3]:
def normalize_hu(image):
    '''
           将输入图像的像素值(-4000 ~ 4000)归一化到0~1之间
       :param image 输入的图像数组
       :return: 归一化处理后的图像数组
    '''
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image > 1] = 1.
    image[image < 0] = 0.
    return image

def get_segmented_lungs(im):
    '''
    对输入的图像进行肺部区域分割，提取有效的肺部区域，用于模型训练
    :param 输入的图像
    :return: 返回分割结果
    '''
    
    binary = im < -200                      # Step 1: 转换为二值化图像
    cleared = clear_border(binary)          # Step 2: 清除图像边界的小块区域
    label_image = label(cleared)            # Step 3: 分割图像
    
    areas = [r.area for r in regionprops(label_image)]  # Step 4: 保留2个最大的连通区域
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:
                       label_image[coordinates[0], coordinates[1]] = 0
    binary = label_image > 0
    
    selem = disk(2)                         # Step 5: 图像腐蚀操作,将结节与血管剥离
    binary = binary_erosion(binary, selem)
    selem = disk(10)                        # Step 6: 图像闭环操作,保留贴近肺壁的结节
    binary = binary_closing(binary, selem)
    edges = roberts(binary)                 # Step 7: 进一步将肺区残余小孔区域填充
    binary = ndi.binary_fill_holes(edges)
    get_high_vals = binary == 0             # Step 8: 将二值化图像叠加到输入图像上
    im[get_high_vals] = -2000
    #print('lung segmentation complete.')
    return im, binary

def out_boundary_check(boundary):
    return boundary[0] < 0 or boundary[1] > ORIGIN_IMAGE_SIZE or boundary[2] < 0 or boundary[3] > ORIGIN_IMAGE_SIZE
    

def get_nodule_pixel(path, sop_id, edge_maps):
    path = os.path.join(path, sop_id + '.dcm')
#     dcm_file = dm.read_file(path) #获取对应的dcm文件
#     pixel_array = dcm_file.pixel_array
    image = SimpleITK.ReadImage(path)
    pixel_array = np.squeeze(SimpleITK.GetArrayFromImage(image))

#     plt.figure()
#     plt.subplots_adjust(right = 2)
#     plt.subplot(2, 3, 1)
#     plt.imshow(pixel_array, cmap="gray")

    im, binary = get_segmented_lungs(pixel_array)
    pixel_array = normalize_hu(pixel_array) 
    
#     plt.subplot(2, 3, 2)
#     plt.imshow(pixel_array, cmap="gray")

    roi_xy = []
    for edge_map in edge_maps:
        x = int(edge_map.find(NAMESPACE + 'xCoord').text)
        y = int(edge_map.find(NAMESPACE + 'yCoord').text)
        #pixel_array[y][x] = 1 #画边界！！！
        roi_xy.append([x, y])
    xmax = np.array(roi_xy)[:, 0].max()
    xmin = np.array(roi_xy)[:, 0].min()
    ymax = np.array(roi_xy)[:, 1].max()
    ymin = np.array(roi_xy)[:, 1].min()

    x_centroid = int((xmax + xmin) / 2.)
    y_centroid = int((ymin + ymax) / 2.)  # center point
    
    margin = 20
    if out_boundary_check((y_centroid - margin, y_centroid + margin, x_centroid - margin, x_centroid + margin)):
        print("OUT OF BOUNDARY!")
        return None
    figure = np.array(pixel_array)[y_centroid - margin : y_centroid + margin, x_centroid - margin : x_centroid + margin]
#     print("boundary is {} {} {} {}".format(y_centroid - margin, y_centroid + margin, x_centroid - margin, x_centroid + margin))
#     plt.subplot(2, 3, 3)
#     plt.imshow(figure, cmap ='gray')
    
    ret,thresh = cv2.threshold(figure,0.1,1,cv2.THRESH_BINARY)
    thresh = np.uint8(thresh)
    thresh = baweraopen(thresh, 10)
#     plt.subplot(2, 3, 4)
#     plt.imshow(thresh, cmap ='gray')
    
    for i in range(len(thresh)):
        for j in range(len(thresh[0])):
            if thresh[i][j] == 0:
                figure[i][j] = 0
            if thresh[i][j] == 1:
                if i < margin - (y_centroid - ymin) or i > margin + (ymax - y_centroid) \
                    or j < margin - (x_centroid - xmin) or j > margin + (xmax - x_centroid):
                        figure[i][j] = 0
    
#     plt.subplot(2, 3, 5)
#     plt.imshow(figure, cmap ='gray')
    
    
    img_GaussianBlur=cv2.GaussianBlur(figure,(7,7),0)
#     plt.subplot(2, 3, 6)
#     plt.imshow(img_GaussianBlur, cmap='gray')
    
#     plt.show()
    
    return img_GaussianBlur


radioist_node_num = [[0,0], [0,0], [0,0], [0,0]]


def make_dirs(path):
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path)

        
class NoduleInfo:
    def __init__(self, image_array, z_pos):
        self.image_array = image_array
        self.z_pos = z_pos
    
        
def xml_parser(path, dir_path):
    tree = ET.parse(path)
    root = tree.getroot()
    num = 0
    
    for node in root.iter(NAMESPACE + 'readingSession'): #遍历四个专家的诊断信息
        num += 1
        # print(node.tag, num)
        current_radio_path = "Radiologist_" + str(num)
        current_benign_img_output_path = os.path.join(OUTPUT_BENIGN_IMG_PATH, current_radio_path)
        current_benign_npy_output_path = os.path.join(OUTPUT_BENIGN_NPY_PATH, current_radio_path)
        current_malignancy_img_output_path = os.path.join(OUTPUT_MALIGNANCY_IMG_PATH, current_radio_path)
        current_malignancy_npy_output_path = os.path.join(OUTPUT_MALIGNANCY_NPY_PATH, current_radio_path)
#         current_output_path = os.path.join(OUTPUT_IMG_PATH, current_radio_path)
#         current_npy_output_path = os.path.join(OUTPUT_NPY_PATH, current_radio_path)
        
        if node.find(NAMESPACE + "unblindedReadNodule"):
            for nodule in node.iter(NAMESPACE + "unblindedReadNodule"): #遍历每个专家的对肺节点的诊断信息，nodule代表肺结点ELement
                if nodule.find(NAMESPACE + "characteristics"): #如果含有characteristics，则为实验数据要挑选的>3mm的肺结节
                    for characteristic in nodule.iter(NAMESPACE + "characteristics"): #characteristics的各个属性
                        malignancy = characteristic.find(NAMESPACE + "malignancy")
                        if int(malignancy.text) != 3: #malignancy!=4 就可以认定对应的nodule为良性或恶性肺结节
                            #将良恶性肺结节的信息输出到OUTPUT文件夹中
                            if int(malignancy.text) >= 4:
                                flag = 0
                                current_output_path = current_malignancy_img_output_path
                                current_npy_output_path = current_malignancy_npy_output_path
                                print("LOADING malignancy nodule......")
                            else:
                                flag = 1
                                current_output_path = current_benign_img_output_path
                                current_npy_output_path = current_benign_npy_output_path
                                print("LOADING benign nodule......")
                            
                            radioist_node_num[num - 1][flag] += 1
                            rois = nodule.findall(NAMESPACE + 'roi')
                            roi_num = len(list(rois))
                            img_list = []
                                
                            for roi in rois: #遍历所有的roi
                                if roi.find(NAMESPACE + "inclusion").text == "TRUE":
                                    dcm_sop_id = roi.find(NAMESPACE + 'imageSOP_UID') #获取dcm文件的ID号
                                    z_pos = roi.find(NAMESPACE + 'imageZposition')
                                    z_pos = float(z_pos.text)
                                    edge_maps = roi.findall(NAMESPACE + 'edgeMap')
                                    nodule_img = get_nodule_pixel(dir_path, dcm_sop_id.text, edge_maps)
                                    if nodule_img is None:
                                        print("THE BOUNDARY error dir is ", dir_path)
                                        continue
                                    nodule_img = nodule_img * 255
                                    img_list.append(NoduleInfo(nodule_img, z_pos))
                            
                            if len(img_list) == 0: #如果不含有符合标准的40*40结节图像
                                continue
                                
                            img_list.sort(key=lambda x:x.z_pos, reverse=False)
                            img_num = 0

                            array_list = []

                            for img in img_list:
                                array_list.append(img.image_array)
                            
                            zero_image = np.zeros((40, 40))
                            array_num = len(array_list)
                            if array_num < 8:
                                if array_num % 2 == 0:
                                    head = tail = 4 - int(array_num / 2)
                                else:
                                    head = 4 - int(array_num / 2)
                                    tail = 3 - int(array_num / 2)
                                for i in range(head):
                                    array_list.insert(0, zero_image)
                                for i in range(tail):
                                    array_list.insert(len(array_list), zero_image)
                                # print("The image number is {}. So we should INCREASE: {} head {} tail".format(array_num, head, tail))
                            elif len(array_list) > 8:
                                mid_img = int(array_num / 2)
                                temp_list = []
                                for i in range(mid_img - 4, mid_img + 4):
                                    temp_list.append(array_list[i])
                                array_list = temp_list
                                # print("The image number is {}. So we should DECREASE from the two side of {}.".format(array_num, mid_img))
                                
                            nodule_array3D = np.stack(array_list, axis=0) #3D np.array
                            nodule_array3D = nodule_array3D[None, :]
                            npy_output_path = os.path.join(current_npy_output_path, "nodules_" + str(radioist_node_num[num - 1][flag]) + '.npy')
                            np.save(npy_output_path, nodule_array3D)
                            
#                             tt = np.load(npy_output_path)
#                             print(tt.shape)

                            # print(nodule_array3D.shape)
    
                            nodule_output_path = os.path.join(current_output_path, "nodules_" + str(radioist_node_num[num - 1][flag]))
                            make_dirs(nodule_output_path)
                            for i in range(8):
                                img_num += 1
                                img_path = os.path.join(nodule_output_path, str(img_num) + '.png')
                                cv2.imwrite(img_path, array_list[i])
                                


                            
def study_retrieval(path):
    for i in os.listdir(path):
        if not i.startswith('.'): #忽略.DS文件
            i = os.path.join(path, i)
            xml_file, is_CT_file = series_retrieval(i)
            if not is_CT_file: #如果series文件夹下某一DCM为其他Modality，则删除整个文件夹
                #shutil.rmtree(i)
                continue
            #如果是CT影像，则获得series_i中的.xml文件，并进行解析
            if xml_file:
                xml_parser(xml_file, i)
                print("XML FILE parser done -->{}".format(xml_file))
            else:
                global xml_lack_num
                xml_lack_num += 1
                print("XML FILE does not exist. Total: {}".format(xml_lack_num))

In [4]:
if os.path.exists(OUTPUT_PATH):
    shutil.rmtree(OUTPUT_PATH)

make_dirs(OUTPUT_PATH)
OUTPUT_BENIGN_IMG_PATH = os.path.join(OUTPUT_PATH, 'nodule_img/benign/')
OUTPUT_BENIGN_NPY_PATH = os.path.join(OUTPUT_PATH, 'nodule_npy/benign/')

OUTPUT_MALIGNANCY_IMG_PATH = os.path.join(OUTPUT_PATH, 'nodule_img/malignancy/')
OUTPUT_MALIGNANCY_NPY_PATH = os.path.join(OUTPUT_PATH, 'nodule_npy/malignancy/')

make_dirs(OUTPUT_BENIGN_IMG_PATH)
make_dirs(OUTPUT_BENIGN_NPY_PATH)
make_dirs(OUTPUT_MALIGNANCY_IMG_PATH)
make_dirs(OUTPUT_MALIGNANCY_NPY_PATH)

Radiologist_Number = 4
patient_num = 0

for i in range(Radiologist_Number):
    radio_path = "Radiologist_" + str(i + 1)
    make_dirs(os.path.join(OUTPUT_BENIGN_IMG_PATH, radio_path))
    make_dirs(os.path.join(OUTPUT_MALIGNANCY_IMG_PATH, radio_path))
    make_dirs(os.path.join(OUTPUT_BENIGN_NPY_PATH, radio_path))
    make_dirs(os.path.join(OUTPUT_MALIGNANCY_NPY_PATH, radio_path))

for patient in patients: #遍历所有的patient——file指代每一个patient文件夹
    if patient.startswith('.'): #忽略.DS文件
        continue
    if patient == "LIDC-IDRI-0659" or patient == "LIDC-IDRI-0365" or patient == "LIDC-IDRI-0017" or patient == "LIDC-IDRI-0566": #365、659号病人存在医师SOPID标注错误的情况
        print("SOP ID Error!")
    else:
        patient = os.path.join(ROOT_PATH, patient) #路径拼接
        studies = os.listdir(patient)
        for study in studies: #遍历patient文件夹——study指代每一个study文件夹
            if not study.startswith('.'): #忽略.DS文件
                study = os.path.join(patient, study)
                study_retrieval(study)
    patient_num += 1
    print("{} of 1010 have done.".format(patient_num))
    print("----------")

print("All the nodules have been detected.")
print("Thanks for all the people who give me help!")

#暂时出现四个问题
#1.病人478 出现了thresh mask 为nonetype的情况，原因为截取的40*40图像造成数组越界
#2.病人679 医生标注的SOPID出现问题
#3.有的dcm文件没有BodyPartExamined字段
#4.某个文件夹只存在DCM文件，而对应的XML文件缺失

LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0787/01-01-2000-53104/30821-77077/018.xml
1 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-1003/01-01-2000-84244/3000023-44795/102.xml
2 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0125/01-01-2000-61676/3000702-08261/011.xml
3 of 1010 have done.
----------
XML FILE does not exist. Total: 1
4 of 1010 have done.
----------
LOADING 

XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0711/01-01-2000-28203/2-20976/099.xml
38 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0375/01-01-2000-23383/30472-71780/118.xml
39 of 1010 have done.
----------
XML FILE does not exist. Total: 7
40 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0729/01-01-2000-30297/3000680-03237/081.xml
41 of 1010 have done.
----------
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0943/01-01-2000-66506/30311-27409/162.xml
42 of 1010 have done.
----------
XML FILE does not exist. Total: 8
43 of 1010 have done.
----------
XML FILE does not exist. Total: 9
44

LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0549/01-01-2000-CT LUNG SCREEN-09475/27966/294.xml
72 of 1010 have done.
----------
XML FILE does not exist. Total: 14
73 of 1010 have done.
----------
XML FILE does not exist. Total: 15
74 of 1010 have done.
----------
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0327/01-01-2000-CHEST-77213/31288-Recon 2 ACRIN LARGE-91756/067.xml
75 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0743/01-01-2000-49810/3000027-49911/067.xml
76 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0571/01-01-2000-CT LUNG SCREEN-22867/40520/016.xml
77 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign

LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0726/01-01-2000-02449/1496-NLST TLC VOL B30F-24024/084.xml
104 of 1010 have done.
----------
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0514/01-01-2000-87995/3000281-52230/259.xml
105 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0184/01-01-2000-CT THORAX WCONTRAST-63701/3-Recon 2 CHEST-04872/123.xml
106 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0980/01-01-2000-26562/3152-20813/125.xml
107 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING malignancy nodul

XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0239/01-01-2000-CT THORAX WO CONTRAST-05073/3-Recon 2 CHEST-62398/228.xml
136 of 1010 have done.
----------
XML FILE does not exist. Total: 19
137 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0457/01-01-2000-85915/3000292-45513/201.xml
138 of 1010 have done.
----------
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0665/01-01-2000-CHEST172-42132/32206-Recon 2 ACRIN LARGE-67533/145.xml
139 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0201/01-01-2000-CT THORAX WCONTRAST-32423/4-45994/155.xml
140 of 1010 have done.
----------
LOADING malignan

...

LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0617/01-01-2000-51600/30081-14069/061.xml
862 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0273/01-01-2000-CT THORAX WCONTRAST-57410/2-CHEST-88746/294.xml
863 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0041/01-01-2000-19169/3000568-87015/147.xml
864 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Do

LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0939/01-01-2000-CT THORAX WO IV CONTRA-99879/30930-Recon 2 LOW DOSE THROUGH LUNGS-19490/166.xml
897 of 1010 have done.
----------
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0105/01-01-2000-23262/3000665-78952/277.xml
898 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0595/01-01-2000-50972/5440-89637/039.xml
899 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser d

XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0158/01-01-2000-49056/3000737-58706/071.xml
930 of 1010 have done.
----------
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0964/01-01-2000-CT LUNG SCREEN-54418/86458/141.xml
931 of 1010 have done.
----------
LOADING benign nodule......
LOADING malignancy nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0990/01-01-2000-76086/30042-68974/115.xml
932 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0707/01-01-2000-72307/5919-72447/103.xml
933 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser do

LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0907/01-01-2000-CT CHEST O CONTR-86190/73696/198.xml
965 of 1010 have done.
----------
LOADING benign nodule......
LOADING benign nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0594/01-01-2000-CT CHEST O CONTR-43117/17124/038.xml
966 of 1010 have done.
----------
XML FILE does not exist. Total: 94
967 of 1010 have done.
----------
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0560/01-01-2000-CT LUNG SCREEN-02875/73357/005.xml
968 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0104/01-01-2000-16722/5597-08531/275.xml
969 of 1010 have done.
----------
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
XML FILE parser done -->../..

LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
XML FILE parser done -->../../Downloads/LIDC-IDRI/LIDC-IDRI-0195/01-01-2000-CT THORAX WCONTRAST-75001/3-55750/143.xml
1006 of 1010 have done.
----------
XML FILE does not exist. Total: 99
1007 of 1010 have done.
----------
LOADING benign nodule......
LOADING malignancy nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING benign nodule......
LOADING malignancy nodule......
LOADING benign nodule......
LOADING malignancy nodule......
X